<a href="https://colab.research.google.com/github/INT2-group18/Network/blob/main/INT2-NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torchvision.models as models
import matplotlib.pyplot as plt

In [3]:
from torch.utils.data import random_split

device = (
    "cuda"
    if torch.cuda.is_available() else "cpu"
)
print(f"Using {device} device")

img_size = (300, 300)
transform = transforms.Compose([transforms.Resize(img_size), ToTensor()])
transform2 =  transforms.Compose([transforms.Resize(img_size), transforms.RandomRotation(90), transforms.RandomHorizontalFlip(), transforms.ColorJitter(hue=.05, saturation=.05), ToTensor()])
transform3 = transforms.Compose([transforms.Resize(img_size), ToTensor(), transforms.ColorJitter(hue=.02, saturation=.02), transforms.RandomHorizontalFlip(), transforms.RandomRotation(20), transforms.Grayscale(num_output_channels=3)])
training_data1 = datasets.Flowers102(root="../flowerData", split="train", download=True, transform=transform)
training_data2 = datasets.Flowers102(root="../flowerData", split="train", download=True, transform=transform2)
training_data3 = datasets.Flowers102(root="../flowerData", split="train", download=True, transform=transform2)
training_data4 = datasets.Flowers102(root="../flowerData", split="train", download=True, transform=transform2)
training_data = ConcatDataset([training_data1, training_data2, training_data3, training_data4])
test_data1 = datasets.Flowers102(root="../flowerData", split="test", download=True, transform=transform)
test_data = ConcatDataset([test_data1])
training_data_full = ConcatDataset([training_data1, training_data2, training_data3, training_data4])



# Split the training data into train and validation sets with an 80/20 ratio
train_size = int(0.8 * len(training_data_full))
val_size = len(training_data_full) - train_size
training_data, validation_data = random_split(training_data_full, [train_size, val_size])


Using cuda device


100%|██████████| 344862509/344862509 [00:30<00:00, 11319833.28it/s]


Extracting ../flowerData/flowers-102/102flowers.tgz to ../flowerData/flowers-102


100%|██████████| 502/502 [00:00<00:00, 535624.68it/s]


100%|██████████| 14989/14989 [00:00<00:00, 15119870.77it/s]


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3), stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(3,3), stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=(3,3), stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.relu4 = nn.ReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=(3,3), stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=(2, 2))
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 9 * 9, 1024)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 1024)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(1024, 102)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.pool4(x)
        
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.pool5(x)
        
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        logits = self.fc3(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, t

In [5]:
def train_loop(dataloader, model, loss_fn, optimiser):
    size = len(dataloader.dataset)
    total_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
        total_loss += loss.item()
        #if batch % 200 == 0:
            #loss, current = loss.item(), (batch + 1) * len(X)
            #print(f"loss: {loss:>7f}    [{current:>5d}/{size:>5d}]")
    print(f"Avg train loss: {(total_loss/len(dataloader)):>8f}")

In [6]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    
learning_rate = 0.0005
batch_size = 64
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.1, weight_decay=0)


train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [7]:
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimiser)
    print("Validation performance:")
    test_loop(validation_dataloader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
Avg train loss: 4.669604
Validation performance:
Test Error: 
 Accuracy: 1.3%, Avg loss: 4.620359 

Epoch 2
-------------------------------
Avg train loss: 4.553140
Validation performance:
Test Error: 
 Accuracy: 2.8%, Avg loss: 4.544501 

Epoch 3
-------------------------------
Avg train loss: 4.452853
Validation performance:
Test Error: 
 Accuracy: 4.8%, Avg loss: 4.426299 

Epoch 4
-------------------------------
Avg train loss: 4.368044
Validation performance:
Test Error: 
 Accuracy: 6.1%, Avg loss: 4.368923 

Epoch 5
-------------------------------
Avg train loss: 4.265355
Validation performance:
Test Error: 
 Accuracy: 7.2%, Avg loss: 4.285997 

Epoch 6
-------------------------------
Avg train loss: 4.163451
Validation performance:
Test Error: 
 Accuracy: 7.7%, Avg loss: 4.192699 

Epoch 7
-------------------------------
Avg train loss: 4.074706
Validation performance:
Test Error: 
 Accuracy: 9.9%, Avg loss: 4.102925 

Epoch 8
------------